In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model

In [2]:
import json

# Function to load configurations
def load_config(config_file):
    with open(config_file, 'r') as file:
        config = json.load(file)
    return config

# Load the configuration
config = load_config('config/config.json')

# Access the dataset path
dataset_dir = config['dataset_path']
train_dir = config['train_path']
val_dir = config['val_path']
test_dir = config['test_path']

In [3]:
# Number of species classes (set this according to your dataset, for example 593 classes)
num_species_classes = 20

# Image Data Generators for augmenting and rescaling images
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [4]:
# Load training, validation, and test data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # VGG16 expects 224x224 images
    batch_size=32,
    class_mode='binary'  # Categorical for multi-class classification
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)

Found 194 images belonging to 2 classes.
Found 28 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [5]:
# Load the VGG16 model, excluding the top fully connected layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [6]:
# Freeze the VGG16 layers so they are not updated during training
base_model.trainable = False

# Add custom layers on top of VGG16 for your mushroom classification task
x = base_model.output
x = Flatten()(x)  # Flatten the output from the convolutional layers
x = Dense(512, activation='relu')(x)  # Dense layer with 512 units
x = Dropout(0.5)(x)  # Dropout for regularization to avoid overfitting
x = Dense(256, activation='relu')(x)  # Another dense layer
x = Dropout(0.5)(x)  # Another Dropout layer

# Output layer for binary classification (edible vs. poisonous)
output_edible = Dense(1, activation='sigmoid')(x)  # Sigmoid activation for binary classification

In [7]:
# Create the model
model = Model(inputs=base_model.input, outputs=output_edible)

# Compile the model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # Binary crossentropy for binary classification
    metrics=['accuracy']
)

In [13]:
number_of_epochs = 10

In [8]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=10  # Adjust the number of epochs based on your dataset size
)

Epoch 1/10


/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6/6 ━━━━━━━━━━━━━━━━━━━━ 20s 3s/step - accuracy: 0.5679 - loss: 1.7918 - val_accuracy: 0.5714 - val_loss: 1.6109
Epoch 2/10
1/6 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.5000 - loss: 4.4352

2024-11-05 21:51:51.867396: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/opt/anaconda3/envs/Thesis/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 515ms/step - accuracy: 0.5000 - loss: 4.4352 - val_accuracy: 0.5714 - val_loss: 1.9976
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5292 - loss: 2.9380 - val_accuracy: 0.4286 - val_loss: 2.8454
Epoch 4/10
1/6 ━━━━━━━━━━━━━━━━━━━━ 9s 2s/step - accuracy: 0.5312 - loss: 3.3175

2024-11-05 21:52:13.087715: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 549ms/step - accuracy: 0.5312 - loss: 3.3175 - val_accuracy: 0.4643 - val_loss: 0.7847
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.5149 - loss: 3.6399 - val_accuracy: 0.4286 - val_loss: 1.6847
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 513ms/step - accuracy: 0.5625 - loss: 2.4802 - val_accuracy: 0.4286 - val_loss: 3.1994
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 19s 2s/step - accuracy: 0.5525 - loss: 2.9559 - val_accuracy: 0.4286 - val_loss: 1.3763
Epoch 8/10
1/6 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - accuracy: 0.5000 - loss: 2.0260

2024-11-05 21:52:56.548622: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 565ms/step - accuracy: 0.5000 - loss: 2.0260 - val_accuracy: 0.6071 - val_loss: 0.6491
Epoch 9/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 17s 3s/step - accuracy: 0.5429 - loss: 2.3605 - val_accuracy: 0.6071 - val_loss: 0.7562
Epoch 10/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 551ms/step - accuracy: 0.4688 - loss: 2.6682 - val_accuracy: 0.6071 - val_loss: 0.6385


In [9]:
# Optionally, unfreeze some layers in the base model and fine-tune
base_model.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# Fine-tune the model
history_finetune = model.fit(
    train_generator,
    steps_per_epoch = train_generator.samples // train_generator.batch_size,
    validation_data = val_generator,
    validation_steps = val_generator.samples // val_generator.batch_size,
    epochs = 5  # Fine-tune for a few more epochs
)

Epoch 1/5


/opt/anaconda3/envs/Thesis/lib/python3.10/site-packages/keras/src/losses/losses.py:27: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 42s 6s/step - accuracy: 0.6458 - loss: 0.0000e+00 - val_accuracy: 0.4286 - val_loss: 0.0000e+00
Epoch 2/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 535ms/step - accuracy: 0.7500 - loss: 0.0000e+00 - val_accuracy: 0.4286 - val_loss: 0.0000e+00
Epoch 3/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 47s 7s/step - accuracy: 0.5813 - loss: 0.0000e+00 - val_accuracy: 0.4286 - val_loss: 0.0000e+00
Epoch 4/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 3s 518ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.4286 - val_loss: 0.0000e+00
Epoch 5/5
6/6 ━━━━━━━━━━━━━━━━━━━━ 44s 6s/step - accuracy: 0.5951 - loss: 0.0000e+00 - val_accuracy: 0.4286 - val_loss: 0.0000e+00


In [14]:
# Save the model, optional
model.save(f'Pth_Files/VGG15_{number_of_epochs}.keras')

In [11]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

ValueError: Must provide at least one structure